# Using subspaces

Often times, we encounter operators that have some conservation law. `dynamite` can take advantage of certain conservation laws by working in a restricted subspace.

Here's a quick summary in table form of the subspaces implemented in `dynamite`, and the operator whose eigenspaces define the subspace:

|    Subspace    |                 Operator                |
|----------------|-----------------------------------------|
| `SpinConserve` | $\sum_i S^z_i$ (total magnetization)    |
| `Parity`       | $\prod_i Z_i$ (parity of # of up spins) |
| `XParity`      | $\prod_i X_i$                           |
| `Explicit`     | (user-defined)                          |
| `Auto`         | (user-defined)                          |

## SpinConserve

The first conservation law we will look at is conservation of total magnetization. One of our favorite models with this symmetry is the Heisenberg model:

$$H = \sum_{\langle i,j \rangle} \vec{S}_i \cdot \vec{S}_j$$

where $\vec{S} = (S^x, S^y, S^z)$. 

Let's implement it:

In [ ]:
# always a good idea to set the spin chain length globally
from dynamite import config
config.L = 24

In [ ]:
from dynamite.operators import sigmax, sigmay, sigmaz, index_sum

def heisenberg():
    paulis = [sigmax, sigmay, sigmaz]
    return index_sum(sum(0.25*p(0)*p(1) for p in paulis))  # 0.25 to account for the factors of 1/2 in each spin operator!

# let's run the function and see what we get
H = heisenberg()
H

Looks pretty good! What is the dimension of the matrix corresponding to this operator?

In [ ]:
H.dim

A dimension $2^{24} \sim 16$ million square matrix. Even though it's sparse, storing that matrix would probably make your laptop pretty unhappy. 

(You may be wondering: why isn't my laptop *already* unhappy, didn't I just build the matrix? In fact no, dynamite delays building the matrix until it needs to, for example when you use it in a computation).

Let's see what happens if we instead switch to working in the total magnetization conserving subspace:

In [ ]:
from dynamite.subspaces import SpinConserve

In [ ]:
# L is total spins, k is the number of "up" spins
# here we work in the half filling symmetry sector 
H.subspace = SpinConserve(L=config.L, k=config.L//2)
H.dim

The dimension has been reduced by a factor of more than 6! In fact, it has been reduced to 24 choose 12, which is what we would expect for total spin conservation:

In [ ]:
from scipy.special import binom
binom(24, 12)

Note that subspaces can be applied to State objects in addition to operators:

In [ ]:
from dynamite.states import State

In [ ]:
psi = State(state='U'*12 + 'D'*12,  # first half spins up, second half down
            subspace=SpinConserve(L=config.L, k=config.L//2))
len(psi)

Note that the product state we specify must be in the subspace:

In [ ]:
try:
    # this causes an error
    psi = State(state='U'*11 + 'D'*13,  # first 11 spins up
                subspace=SpinConserve(L=config.L, k=config.L//2))
except ValueError as e:
    print(e)

### The `XParity` subspace

We are not done yet: the Heisenberg model has an *additional* $\mathbb{Z}_2$ (spin flip) symmetry in addition to the total magnetization conservation. We can apply the `XParity` subspace on top of `SpinFlip` to yield an even smaller subspace:

In [ ]:
from dynamite.subspaces import XParity
subspace = XParity(
    SpinConserve(L=config.L, k=config.L//2),
    sector='+'  # '+' means that we are in the symmetric symmetry sector, '-' for antisymmetric
)

subspace.get_dimension()  # half as big as before!

Note that `XParity` is the *only* subspace that can be applied on top of another one. The basis states of the other subspaces are product states (in the $Z$ basis)---they make the Hilbert space smaller by simply ignoring product states in other symmetry sectors. `XParity`, however, is the only subspace whose basis states are *not* product states. It combines two product states in the parent subspace into a single basis state, of the form

$$\left|\psi_c\right> = \left|c\right> \pm \left|\bar c\right>$$

where $\left|c\right>$ is a product state in the $Z$ basis, and $\left|\bar c\right>$ is the complement (all spins flipped) of that state. The sign in between the two states is controlled by the `sector` argument to `XParity`.

In [ ]:
# exercise: create a state in the XParity SpinConserve subspace
# and confirm that the global spin-flip operator has the expected eigenvalue 
# (+1 for the symmetric subspace, -1 for antisymmetric)

# it's easiest to set the subspace globally so it will be applied to both
# the spin-flip operator and the state you create. do it like this:
# config.subspace = ... TODO ...

# here is the global spin-flip operator, to get you started
from dynamite.operators import index_product
global_spinflip_operator = index_product(sigmax())

# your code for computing the expectation value (eigenvalue in this case) here!
# don't forget to initialize your State object to something (a product state, or random is fine)


## Parity

The next subspace we'll examine is parity conservation. This means that the total number of up spins is not globally conserved, but is instead conserved mod 2. This is the same conservation law as `XParity`, but in the $Z$ basis instead of the $X$ basis.

For this, we'll use the following long-range XX+Z model:

In [ ]:
config.subspace = None  # clear out any leftover configured subspace from above

from dynamite.operators import sigmax, sigmaz, index_sum, op_sum

def XXZ():
    interaction = op_sum(index_sum(sigmax(0)*sigmax(i)) for i in range(1, config.L))
    uniform_field = 0.5*index_sum(sigmaz())
    return interaction + uniform_field

# look at an example. we still have L=24 set from above
XXZ()

We can see by inspection of the Hamiltonian's definition that the X terms are always two-body, meaning that parity is conserved in the Z product state basis. We can easily apply this subspace in dynamite:

In [ ]:
H = XXZ()

print('full space dimension:     ', H.dim)

from dynamite.subspaces import Parity
H.subspace = Parity('even')

print('parity subspace dimension:', H.dim)

As expected, the dimension was cut in half. If you want the symmetry sector which has an odd number of up spins, just pass 'odd' to the subspace.

Note that we cannot use `XParity` for this Hamiltonian because it doesn't conserve parity in the $X$ basis. But we could re-write the Hamiltonian to be ZZ+X instead of XX+Z, in which case `XParity` would work. (Feel free to try this if you'd like!)

## Explicit

The last subspace that we will discuss here is the Explicit subspace. This allows you to define your own basis of product states! You could you this, for example, to define a Rydberg basis in which no up spin is adjacent to another up spin, for some aribtrary connectivity.

Under the hood, dynamite represents product states as integers, where each bit of the integer represents a spin (0=up, 1=down). The least significant bit of the integer is spin 0. This is how we will pass our array of product states to the Explicit subspace.

As a simple example here, we will reproduce the total magnetization conservation subspace by hand!

In [ ]:
# let's use numpy's function that counts the number of set bits in an integer,
# to pick out all integers between 0 and 2^L-1 that have L//2 bits set to 1 
import numpy as np

# note: this toy example is very slow, you probably want to do something more clever 
# with e.g. numpy vectorization when creating your own list of states for Explicit subspaces!
config.L = 18
subspace_product_states = [x for x in range(2**config.L) if np.int64(x).bit_count() == config.L//2]

# show that these states all have half spins up:
def print_first_states(state_list):
    for x in state_list[:10]:
        print(bin(x)[2:].zfill(config.L)) # print binary representation, with leading zeros
    print('...')
    
print_first_states(subspace_product_states)

Now that we have our list of product states for the basis, we simply call the Explicit subspace:

In [ ]:
from dynamite.subspaces import Explicit

In [ ]:
explicit_subsp = Explicit(subspace_product_states)

# it's equivalent to the SpinConserve one! just made by hand
explicit_subsp == SpinConserve(config.L, config.L//2)

## Auto

Lastly, let's take a look at the `Auto` subspace. 
It's actually a subclass of `Explicit`, but in this case the user isn't responsible for supplying the list of product states. 
Instead, it's determined automatically from a given Hamiltonian and product state. 
Under the hood, `Auto` performs a breadth-first-search through the graph of product states for which the Hamiltonian is an adjacency matrix---thus automatically determining which subset of product states are in the same symmetry sector. 

As an example, we show yet another way of reproducing the `SpinConserve` subspace: by applying `Auto` to the Heisenberg model we defined at the beginning of this notebook!

In [ ]:
from dynamite.subspaces import Auto

half = config.L // 2
init_state = 'U'*half + 'D'*half
auto_subsp = Auto(heisenberg(), init_state)

# check out the first few states
print_first_states(auto_subsp.idx_to_state(np.arange(10)))

In [ ]:
# explicitly check that this again reproduces the SpinConserve subspace
auto_subsp == SpinConserve(config.L, config.L//2)

By default, `Auto` sorts the list of states it computes via the breadth-first search. However, the sorting can be turned off, in which case `Auto` directly uses the ordering resulting from the breadth-first search.
This ordering is called [Cuthill-McKee](https://en.wikipedia.org/wiki/Cuthill%E2%80%93McKee_algorithm) and can improve the performance of certain algorithms, such as the shift-invert spectral transformation used by dynamite to solve for eigenpairs in the middle of the spectrum.

In [ ]:
unsorted_auto_subsp = Auto(heisenberg(), init_state, sort=False)
print_first_states(unsorted_auto_subsp.idx_to_state(np.arange(10)))

## Up next

[Continue to notebook 6](6-ShellMatrices.ipynb) to learn how to save memory through "matrix-free" computation. 